<a href="https://colab.research.google.com/github/codedyasai/Python_MachineLearning/blob/main/08_%EA%B5%90%EC%B0%A8%EA%B2%80%EC%A6%9D%EA%B3%BC_%EA%B7%B8%EB%A6%AC%EB%93%9C%EC%84%9C%EC%B9%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 모듈 로딩

In [ ]:
from IPython.display import display
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
!pip install mglearn
import mglearn

# 음수표현 라이브러리
plt.rcParams['axes.unicode_minus'] = False

# 경고무시
import warnings
warnings.filterwarnings("ignore")

# 매직명령어 : 시각화 결과가 노트북에 포함되도록
%matplotlib inline

# 교차 검증과 그리드 서치

- 검증세트와 교차검증
- 그리드 서치와 랜덤서치

In [ ]:
wine = pd.read_csv('https://raw.githubusercontent.com/rickiepark/hg-mldl/master/wine.csv')
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [ ]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

## 훈련, 테스트, 검증세트 나누기

In [ ]:
from sklearn.model_selection import train_test_split

# 훈련세트와 테스트세트
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size= 0.2, random_state= 42)

# 훈련세트를 훈련세트와 검증세트로 나눔
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size= 0.2,random_state= 42)

In [ ]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


## 모델링
- 검증세트로 모델 평가

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt= DecisionTreeClassifier(random_state= 42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

# 훈련데이터의 양이 너무 적었다.

0.9971133028626413
0.864423076923077


## 교차검증: k-폴드 교차 검증
- 훈련 세트를 k개의 구분으로 나누어서 교차검증을 수행하는 것
- cross_validate() 교차검증 함수 사용, 기본값 cv 매개변수= 5

In [ ]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
# fit_time, score_time, test_score
print(scores)

{'fit_time': array([0.02608347, 0.02506113, 0.04806304, 0.02417374, 0.01004934]), 'score_time': array([0.00423288, 0.00178194, 0.00822496, 0.00164771, 0.0015676 ]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


- test score의 평균을 구해 최종 점수를 구하는 것

In [ ]:
np.mean(scores['test_score'])

0.855300214703487

- train_test_split 함수는 훈련세트를 분리할 때 섞어서 분리하지만, cross_validate 함수는 훈련세트를 섞어서 폴드를 나누지 않는다.
- 회귀모델 KFold 분할기, 분류모델 StratifiedKFold 분할기 사용

In [ ]:
from sklearn.model_selection import StratifiedGroupKFold

scores = cross_validate(dt, train_input, train_target, cv= StratifiedGroupKFold())
# fit_time, score_time, test_score
print(np.mean(scores['test_score']))

nan


In [ ]:
# 훈련세트를 섞고 10-폴드 교차검증을 수행해보자
splitter = StratifiedGroupKFold(n_splits= 10, shuffle= True, random_state= 42)
scores = cross_validate(dt, train_input, train_target, cv= splitter)
# fit_time, score_time, test_score
print(np.mean(scores['test_score']))

nan


# 하이퍼파라미터 튜닝

- 하이퍼파라미터: 사용자가 지정해야만 하는 파라미터
- 파라미터: 모델이 학습을 통해 취득한 파라미터
- 모델파라미터: 모델을 학습한 후 취득하게 되는 파라미터, ex) 속성명_
- 하이퍼파라미터 튜닝방법:
  - 먼저 라이브러리가 제공하는 기본값으로 모델 훈련
  - 검증세트의 점수나 교차검증을 통해 매개변수를 조금씩 바꿔본다
  - 모델마다 적게는 1-2개에서 많게는 5-6개의 매개변수가 제공된다.
  - 모든 파라미터를 다 쓰는 것은 오히려 비효율적이다

# 그리드 서치

- sklearn.model_selection.GridSearchCV: 시도해볼 하이퍼파라미터들을 지정하면 모든 조합에 대해 교차검증 후 제일 좋은 성능을 내는 하이퍼파라미터 조합을 찾아준다.

- 함수
  - fit(X, y): 학습
  - predict(X): 분류-추론한 class, 회귀-추론한 값
  - predict_proba(X): 분류문제에서 class별 확률을 반환
  - cv_results_ : 파라미터 조합별 평가 결과를 Dictionary로 반환
  - best_params_ : 가장 좋은 성능을 낸 파라미터 조합을 반환
  - best_estimator_ : 가장 좋은 성능을 낸 모델 반환
  - best_score_ : 가장 좋은 점수 반환

- 시도할 하이퍼파라미터와 값들이 너무 많아지면 많은 시간이 소요된다.
  - ex) 1번 파라미터: 10개, 2번 파라미터: 5개, 3번 파라미터: 5개
  - = 10 * 5 * 5 = 250개 조합    

In [ ]:
from sklearn.model_selection import GridSearchCV

# min_impurity_decrease: 노드를 나누기 위한 최소 샘플 수
# max_depth: 트리의 깊이
# min_samples_split: 노드를 분리하기 위한 최소 샘플 수
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [ ]:
# n_jobs: 병렬 실행에 사용할 CPU 코어수를 지정하는 것(기본값)
# n_jobs= -1 : 시스템에 있는 모든 코어를 사용한다
gs = GridSearchCV(dt, params, n_jobs= -1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [ ]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [ ]:
gs.best_params_

{'min_impurity_decrease': 0.0001}

In [ ]:
gs.cv_results_

{'mean_fit_time': array([0.03529344, 0.04181137, 0.03304071, 0.0283917 , 0.03122058]),
 'std_fit_time': array([0.00516623, 0.00950471, 0.00809347, 0.00673367, 0.00583476]),
 'mean_score_time': array([0.00292602, 0.01111054, 0.00351276, 0.00482345, 0.00178614]),
 'std_score_time': array([0.00282326, 0.00581788, 0.00218544, 0.00417354, 0.00059744]),
 'param_min_impurity_decrease': masked_array(data=[0.0001, 0.0002, 0.0003, 0.0004, 0.0005],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'min_impurity_decrease': 0.0001},
  {'min_impurity_decrease': 0.0002},
  {'min_impurity_decrease': 0.0003},
  {'min_impurity_decrease': 0.0004},
  {'min_impurity_decrease': 0.0005}],
 'split0_test_score': array([0.86923077, 0.87115385, 0.86923077, 0.86923077, 0.86538462]),
 'split1_test_score': array([0.86826923, 0.86346154, 0.85961538, 0.86346154, 0.86923077]),
 'split2_test_score': array([0.8825794 , 0.87680462, 0.87584216, 0.88161

In [ ]:
gs.cv_results_['mean_test_score']

array([0.86819297, 0.86453617, 0.86492226, 0.86780891, 0.86761605])

In [ ]:
params = {
    'max_depth' : range(5,20,1),
    'min_samples_split':range(2,100,10),
    'min_impurity_decrease':np.arange(0.0001, 0.001, 0.0001)
}

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input,train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [ ]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [ ]:
np.max(gs.cv_results_['mean_test_score'])
gs.cv_results_

0.8683865773302731

In [ ]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
gs.cv_results_['params'][best_index]

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}

## 랜덤 서치(Random Search)

- 매개변수의 값의 목록을 전달하는 것이 아니라 매개변수를 샘플링할 수 있는 확률 분포 객체를 전달
- form scipy.stats import uniform, randint
- uniform: 균등분포 샘플링
- 테스트 세트의 점수는 검증세트에 대한 점수보다 조금 낮은 것이 일반적이다.

In [ ]:
from scipy.stats import uniform, randint

In [ ]:
rgen = randint(0, 10)
rgen.rvs(10)

array([7, 7, 5, 4, 0, 9, 7, 1, 0, 2])

In [ ]:
np.unique(rgen.rvs(1000), return_counts= True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 97, 105, 103,  90, 110,  90, 109, 102, 100,  94]))

In [ ]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.10164636, 0.51792536, 0.57041455, 0.08875587, 0.31671829,
       0.27732322, 0.26752577, 0.26823055, 0.34447783, 0.35512695])

In [ ]:
params = {
    'max_depth': randint(20, 50),
    'min_samples_split': randint(2,25),
    'min_impurity_decrease': uniform(0.0001, 0.001),
    'min_samples_leaf': randint(1, 25)
    }

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(DecisionTreeClassifier(random_state= 42),
                        params, n_iter= 100, n_jobs= -1, random_state= 42)

rs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f231f24ae30>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f231f24b1f0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f231f24b2e0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f231f24b160>},
                   random_state=42)

In [ ]:
rs.best_params_

{'max_depth': 39,
 'min_impurity_decrease': 0.00034102546602601173,
 'min_samples_leaf': 7,
 'min_samples_split': 13}

In [ ]:
# 최고 교차검증 점수
np.max(rs.cv_results_['mean_test_score'])

0.8695428296438884

In [ ]:
dt = rs.best_estimator_
dt.score(test_input, test_target)

0.86